<a href="https://colab.research.google.com/github/josephmailil1/Speciale---Stibo/blob/main/Agent%20Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('OpenAI_Key') #use the name of your colab key

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/My Drive/Speciale")

## Load data

In [ ]:
shap = pd.read_csv("shap_noincome.csv")
data = pd.read_csv("basedata.csv")

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0, openai_api_key=openai_api_key)

In [ ]:
from langchain_experimental.tools import PythonAstREPLTool

In [ ]:
tools = [PythonAstREPLTool(locals={"data": data, "shap": shap})]

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain.output_parsers import JsonOutputToolsParser

In [ ]:
parser = JsonOutputToolsParser(tool = tools)

# Prompt Template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""You have access to a pandas dataframe `data`. \
            This is the final dataframe that was used to predict Income with, using a Random Forest ML model. \
            The dataset is a publicly available dataset called `adult census`, where prediction task is to determine whether a person makes over $50K a year. \

            You are also given the pandas dateframe `shap`. This dataframe contains shap values for the test set obsevations. \
            The shap obsevation shows which values are important for the final prediction of each local prediction.

            If asked about the importance/relevance of obsevations or features, use the `shap` dataframe to answer. \
            A high absolute value of the shap dataset means that the value is relevant. \
            If a prediction is made on race, sex, native-country or age please highlight the possibility that the individual is a victim of discrimination \

            Use the `data` dataframe to answer question about what the input values to the obsevation is. \
            Both Dataframe start at index 0.

            Respond directly to the question once you have enough information to answer it. \
            Please add context to your answers, and try to explain the implications of the result.""",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Agent

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps = False)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def call_agent(Question):
    answer = agent_executor.invoke({"input": Question})
    return answer['output']

# Gradio

In [ ]:
pip install gradio

In [ ]:
import gradio as gr

In [ ]:
demo = gr.Interface(
    fn=call_agent,
    inputs=["text"],
    outputs=[gr.Text(label = "Answer")],
    theme = "Soft",
    title = 'Adult Census Explanation',
    description = "Explanation of Random Forest Prediction Modelling and SHAP values for local interpretation",
    examples=["Explain the 1st obsevation", "What is the model accuracy", "What are the reason for the 58th obsevation"],
    allow_flagging = "never"
)

In [ ]:
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://edd97a874538574958.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
